In [2]:
import pandas as pd
import sqlite3
from datetime import datetime

In [3]:
# ------------------------------
# Load CSV
# ------------------------------
payments_df = pd.read_csv("csv_data/tbl_accounts.csv")

In [4]:
# Normalize lowercase
payments_df = payments_df.applymap(lambda x: str(x).lower() if pd.notnull(x) else None)

C:\Users\balot\AppData\Local\Temp\ipykernel_14400\1720972424.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payments_df = payments_df.applymap(lambda x: str(x).lower() if pd.notnull(x) else None)


In [5]:
# ------------------------------
# Connect to DB
# ------------------------------
conn = sqlite3.connect("sqlite_db/guide-mts-data.sqlite3")
cursor = conn.cursor()

In [7]:
# Load work_descriptions for mapping
work_desc_df = pd.read_sql("SELECT id, customer_id, work FROM work_descriptions", conn)

In [11]:
# Load jobs to map jobid -> jobdesc
jobs_df = pd.read_csv("csv_data/mst_job.csv")
jobs_df = jobs_df.applymap(lambda x: str(x).lower() if pd.notnull(x) else None)

C:\Users\balot\AppData\Local\Temp\ipykernel_14400\1285410995.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  jobs_df = jobs_df.applymap(lambda x: str(x).lower() if pd.notnull(x) else None)


In [14]:
# ------------------------------
# Merge logic
# ------------------------------
# Step 1: Map jobid -> jobdesc
payments_df = payments_df.merge(jobs_df[["JobID", "JobDesc"]], on="JobID", how="left")

# Step 2: Match with work_descriptions (customer_id + jobdesc)
# Ensure both keys are of the same type (string)
work_desc_df['customer_id'] = work_desc_df['customer_id'].astype(str)

merged = payments_df.merge(
    work_desc_df,
    left_on=["StuID", "JobDesc"],
    right_on=["customer_id", "work"],
    how="inner"
)

In [15]:
# ------------------------------
# Prepare final insert DF
# ------------------------------
final_df = pd.DataFrame()
final_df["customer_id"] = merged["StuID"]
final_df["work_desc_id"] = merged["id"]
final_df["payment_mode"] = merged["ModeOfPymt"]
final_df["amount_paid"] = merged["PaidAmt"]
final_df["created_on"] = merged["PymtDt"]

# current timestamp
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
final_df["updated_on"] = current_time

In [16]:
# ------------------------------
# Insert into SQLite
# ------------------------------
final_df.to_sql("payments", conn, if_exists="append", index=False)

print("✅ Payments inserted successfully into payments table")
print(final_df.head(10))

✅ Payments inserted successfully into payments table
  customer_id  work_desc_id payment_mode amount_paid           created_on  \
0          53            28         cash         300  2013-05-20 00:00:00   
1          54            23       select           0  2013-05-20 00:00:00   
2          46            21         cash        3500  2013-05-20 00:00:00   
3          47            91         cash        3500  2013-05-20 00:00:00   
4          51             9         cash        3500  2013-05-20 00:00:00   
5          43            48         cash        3500  2013-05-20 00:00:00   
6          42            47         cash        3500  2013-05-20 00:00:00   
7          54            23         cash        3500  2013-05-20 00:00:00   
8          55            15       select           0  2013-05-20 00:00:00   
9          56            20       select           0  2013-05-21 00:00:00   

            updated_on  
0  2025-08-24 12:51:08  
1  2025-08-24 12:51:08  
2  2025-08-24 12:51:08  